In [5]:
import matplotlib.pyplot as plt
import scipy as sp
import numpy as np
import os
import time

from skimage import morphology
from skimage.segmentation import expand_labels, watershed
from scipy.ndimage import convolve1d
from dask_ml.cluster import KMeans
from dask_ml.decomposition import PCA

from dask_image import imread
from dask_image import ndmeasure
from dask_image import ndfilters
from dask import array as da
from dask.distributed import Client, progress

import napari

In [6]:
client = Client(processes=False, threads_per_worker=6,
                n_workers=1, memory_limit='12GB')
client

C:\Users\johnsor\AppData\Local\miniforge3\envs\stat-shape\Lib\site-packages\distributed\node.py:187: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 50448 instead
  warnings.warn(


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://172.19.111.216:50448/status,
Dashboard: http://172.19.111.216:50448/status,Workers: 1
Total threads: 6,Total memory: 11.18 GiB
Status: running,Using processes: False
Comm: inproc://172.19.111.216/10848/1,Workers: 0
Dashboard: http://172.19.111.216:50448/status,Total threads: 0
Started: Just now,Total memory: 0 B
Comm: inproc://172.19.111.216/10848/4,Total threads: 6
Dashboard: http://172.19.111.216:50449/status,Memory: 11.18 GiB
Nanny: None,


In [46]:
#Specific is the name of the folder of the orginal data
specific = 'F231006L'
#which_finger should be 1,2,3, or 4, corresponding to the index, middle, ring, and pinky fingers
which_finger = 3

In [47]:
finger_dir = {1: '../fingers/index/full/' + specific + '/',
              2: '../fingers/middle/full/' + specific + '/',
              3: '../fingers/ring/full/' + specific + '/',
              4: '../fingers/pinky/full/' + specific + '/',
             }
temp_eigs_dir = {1: '../fingers/index/full/' + specific + ' - eigens/',
              2: '../fingers/middle/full/' + specific + ' - eigens/',
              3: '../fingers/ring/full/' + specific + ' - eigens/',
              4: '../fingers/pinky/full/' + specific + ' - eigens/',
             }

In [48]:
#If the file is small enough for your computer, 
#then add ".compute" to the end of the line below below.
finger = da.from_npy_stack(finger_dir[which_finger])
spacing = np.load('../spacing/' + specific + '.npy')

In [49]:
dd_sigma = 2

In [50]:
#The size of the filter is determiend by dd_sigma above
s = dd_sigma
range_1d = np.arange(-4*s, 4*s + 1)
length_1d = 8*s + 1

#1-D Gaussian approximation
G = np.array([1/(np.sqrt(2*np.pi)*s) * np.exp(-x**2/(2*s**2)) for x in range_1d])
G_x = G[np.newaxis, np.newaxis, :]
G_y = G[np.newaxis, :, np.newaxis]
G_z = G[:,np.newaxis, np.newaxis]

#1-D Gaussian derivative approximation
dG = np.array([-x/(np.sqrt(2*np.pi)*s**3) * np.exp(-x**2/(2*s**2)) for x in range_1d])
dG_x = dG[np.newaxis, np.newaxis, :]
dG_y = dG[np.newaxis, :, np.newaxis]
dG_z = dG[:,np.newaxis, np.newaxis]

#1-D Gaussian 2nd derivative approximation
ddG = np.array([(x**2 - s**2)/(np.sqrt(2*np.pi)*s**5) * np.exp(-x**2/(2*s**2)) for x in range_1d])
ddG_x = ddG[np.newaxis, np.newaxis, :]
ddG_y = ddG[np.newaxis, :, np.newaxis]
ddG_z = ddG[:,np.newaxis, np.newaxis]

In [51]:
#Do the convolutions
dxdxGcI = ndfilters.convolve(
    ndfilters.convolve(
        ndfilters.convolve(finger, ddG_x),
        G_y,
    ),
    G_z,
)
dxdyGcI = ndfilters.convolve(
    ndfilters.convolve(
        ndfilters.convolve(finger, dG_x),
        dG_y,
    ),
    G_z,
)
dydyGcI = ndfilters.convolve(
    ndfilters.convolve(
        ndfilters.convolve(finger, G_x),
        ddG_y,
    ),
    G_z,
)
dydzGcI = ndfilters.convolve(
    ndfilters.convolve(
        ndfilters.convolve(finger, G_x),
        dG_y,
    ),
    dG_z,
)
dzdzGcI = ndfilters.convolve(
    ndfilters.convolve(
        ndfilters.convolve(finger, G_x),
        G_y,
    ),
    ddG_z,
)
dzdxGcI = ndfilters.convolve(
    ndfilters.convolve(
        ndfilters.convolve(finger, dG_x),
        G_y,
    ),
    dG_z,
)

In [52]:
DD = np.stack([dxdxGcI, dxdyGcI, dydyGcI, dydzGcI, dzdzGcI, dzdxGcI], axis = 3)

In [53]:
da.to_npy_stack('temp-derivatives/', DD)

In [54]:
DD = da.from_npy_stack('temp-derivatives/')

In [55]:
def viete(the_six):
    if len(the_six) == 6:
        #the_six should be formatted as the [xx, xy, yy, yz, zz, zx] derivatives
        # of the Gaussian convolved with the image
        xx, xy, yy, yz, zz, xz = the_six
        #The characteristic polynomial of the 3x3 matrix will have coefficients:
        trace_av = np.average([xx, yy, zz])
        P_shifted = np.array([[xx-trace_av, xy,          xz],
                              [xy,          yy-trace_av, yz],
                              [xz,          yz,          zz-trace_av]])
        _,_,p,q = np.poly(P_shifted)
        neg_p_sqrt = np.sqrt(-p)
        sqrt_3 = 1.732050807568877
        if p >= -10**(-5): #This will only happen if all off-diagonals are basically zero and
            # all diagonals are basically identical.
            eigens = np.array([trace_av]*3)
        elif 27*q**2 > -4*p**3 - 10**(-5):
            #This can only happen when the discriminant is basically zero
            # which will mean cos(theta) = sign(q).  In either case, cosine of one of
            # the trisected angels will also be sign(q).
            la1 = neg_p_sqrt * 2 * np.sign(q) / sqrt_3 - trace_av
            la23 = neg_p_sqrt * (-np.sign(q)) / sqrt_3 - trace_av
            eigens = np.array(sorted([la1, la23, la23], reverse=True))
        else:
            theta = np.arccos(3*sqrt_3*q/(2*neg_p_sqrt**3))
            cs = np.cos(theta/3)
            sn = np.sqrt(1 - cs**2)
            la1 = neg_p_sqrt * 2 * cs / sqrt_3 - trace_av
            la2 = neg_p_sqrt * (-cs + sqrt_3*sn) / sqrt_3 - trace_av
            la3 = neg_p_sqrt * (-cs - sqrt_3*sn) / sqrt_3 - trace_av
            eigens = np.array(sorted([la1, la2, la3], reverse=True))
        return eigens
    else:
        print(f'Viete: The chosen axis must be of length 6, but instead, got: {the_six} which is length {len(the_six)}')
        return np.zeros(3)

In [56]:
eigs = da.apply_along_axis(viete, 3, DD)

Viete: The chosen axis must be of length 6, but instead, got: [1.] which is length 1


In [57]:
%time da.to_npy_stack(temp_eigs_dir[which_finger], eigs)

CPU times: total: 5h 35min 9s
Wall time: 4h 39min 50s


In [58]:
eigs = da.from_npy_stack(temp_eigs_dir[which_finger])

In [ ]:
eigs